# Comparing global LoCuSS High-$L_{\rm{X}}$ $T_{\rm{X}}$ and $L_{\rm{X}}$ measurements to literature

This notebook measures global temperatures and luminosities for the samples we are using to verify our method, then compares them to the measurements from literature. While the focus of this work is the measurement of galaxy cluster masses, temperatures and luminosities play an important role in the construction of X-ray mass-observable scaling relations - thus they must also be shown to be consistent with previous work. 

## Import Statements

In [1]:
import pandas as pd
import numpy as np
from astropy.units import Quantity, UnitConversionError
from astropy.cosmology import LambdaCDM, WMAP9
import matplotlib.pyplot as plt
from typing import Union, List
from shutil import rmtree
import os

# This adds the directory above to the path, allowing me to import the common functions that I've written in
#  common.py - this just saves me repeating boring code and makes sure its all consistent
import sys
sys.path.insert(0, '..')
from common import locuss_colour, locuss_cosmo, direct_comparison_plot, find_lims, fit_comp

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 80
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/')[0]+'notebooks/xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.samples import ClusterSample
from xga.sources import GalaxyCluster
from xga.sources import BaseSource
from xga.xspec import single_temp_apec

# This is a bit cheeky, but suppresses the warnings that XGA spits out (they are 
#  useful, but not when I'm trying to present this notebook on GitHub)
import warnings
warnings.filterwarnings('ignore')

This subsection involves reading in the sample files of the four test samples (described in [the sample properties notebook](sample_properties.ipynb)), then setting up separate XGA ClusterSample instances (see [the documentation](https://xga.readthedocs.io/en/latest/notebooks/tutorials/sources_samples.html) for an introduction to XGA source and sample objects.

We impose an additional cleaning step on each sample, where we make sure that (for each XMM observation initially associated with a source) at least 70% of a cluster's $R_{500}$ falls on that observation - if this requirement is not met then the observation is excluded. These requirements are set with the `clean_obs=True`, `clean_obs_reg='r500'`, and `clean_obs_threshold=0.7` arguments when a ClusterSample instance is declared.

### LoCuSS High-$L_{\rm{X}}$

The LoCuSS High-$L_{\rm{X}}$ sample was selected from ROSAT for its high luminosity clusters, and will again be a useful comparison as testing against various different analyses is beneficial in establishing the veracity of our new measurements.

In [2]:
locuss = pd.read_csv("../../sample_files/locuss_highlx_clusters.csv", dtype={'chandra_id': str, 'xmm_obsid': str})

In [3]:
# Reading out the relevant values into arrays just for ease of passing into the ClusterSample object
ra = locuss['ra'].values
dec = locuss['dec'].values
z = locuss['z'].values
n = locuss['name'].values
r500 = Quantity(locuss['r500'].values, 'kpc')
r2500 = Quantity(locuss['r2500'].values, 'kpc')


# Declaring the actual ClusterSample instance for the LoCuSS sample
locuss_srcs = ClusterSample(ra, dec, z, n, r500=r500, r2500=r2500, cosmology=locuss_cosmo, load_fits=True, 
                            use_peak=False, clean_obs=True, clean_obs_reg='r500', clean_obs_threshold=0.7)

Declaring BaseSource Sample: 100%|████████████████████████████████████████████| 50/50 [00:21<00:00,  2.35it/s]
Setting up Galaxy Clusters: 50it [00:55,  1.10s/it]                                                           


## Running $T_{\rm{X}}$ and $L_{\rm{X}}$ Measurements

The XGA XSPEC functions that we use here all automatically call the XGA SAS interface, so the necessary spectra are generated before the fits begin. As the different samples measure properties within different spatial regions, all the function calls differ slightly.

The results of the fits are stored within the indivual source objects that make up each sample.

### LoCuSS

The LoCuSS temperatures and luminosities are slightly different, and have been measured with the core of the X-ray emission removed (spectra are generated in the 0.15-1 $R_{500}$ region). This approach has been shown to reduce the scatter in mass with $L_{\rm{X}}$. We can replicate it by setting an inner radius when we call `single_temp_apec`.

In [4]:
single_temp_apec(locuss_srcs, locuss_srcs.r500, inner_radius=0.15*locuss_srcs.r500, lo_en=Quantity(0.7, 'keV'), 
                 hi_en=Quantity(10, 'keV'), freeze_met=False, timeout=Quantity(4, 'hr'))
single_temp_apec(locuss_srcs, locuss_srcs.r500, lo_en=Quantity(0.7, 'keV'), hi_en=Quantity(10, 'keV'), 
                 freeze_met=False, timeout=Quantity(4, 'hr'))

Running XSPEC Fits: 100%|█████████████████████████████████████████████████████| 45/45 [11:01<00:00, 14.69s/it]


## Retrieving $T_{\rm{X}}$ and $L_{\rm{X}}$ measurements from the samples

We must extract and judge the quality of the temperature and luminosity measurements that we have made for each of the samples, then later on we will be able to directly compare them. At the same time we make sure that the results from literature are formatted in such a way that we can easily compare them.

### LoCuSS

In [ ]:
locuss_txce_all = locuss_srcs.Tx(locuss_srcs.r500, inner_radius=0.15*locuss_srcs.r500, quality_checks=False)
locuss_txce = locuss_srcs.Tx(locuss_srcs.r500, inner_radius=0.15*locuss_srcs.r500, quality_checks=True)

locuss_lxbocel_all = locuss_srcs.Lx(locuss_srcs.r500, inner_radius=0.15*locuss_srcs.r500, quality_checks=False, 
                                    lo_en=Quantity(0.01, 'keV'), hi_en=Quantity(100.0, 'keV'))
locuss_lxbolce = locuss_srcs.Lx(locuss_srcs.r500, inner_radius=0.15*locuss_srcs.r500, quality_checks=True, 
                                lo_en=Quantity(0.01, 'keV'), hi_en=Quantity(100.0, 'keV'))

locuss_lxbol_all = locuss_srcs.Lx(locuss_srcs.r500, quality_checks=False, lo_en=Quantity(0.01, 'keV'), 
                                  hi_en=Quantity(100.0, 'keV'))
locuss_lxbol = locuss_srcs.Lx(locuss_srcs.r500,  quality_checks=True, lo_en=Quantity(0.01, 'keV'), 
                              hi_en=Quantity(100.0, 'keV'))

In [ ]:
loclit_tx = []
loclit_lxbol = []
for n in locuss_srcs.names:
    rel_row = locuss[locuss['name'] == n].iloc[0]
    loclit_tx.append(Quantity(rel_row[['x_Tx500ce', 'x_Tx500ce-', 'x_Tx500ce+']], 'keV'))
    loclit_lxbol.append(Quantity(rel_row[['x_Lxbol', 'x_Lxbol_err', 'x_Lxbol_err']]*1e+44, 'erg/s'))
    
loclit_tx = Quantity(loclit_tx)
loclit_lxbol = Quantity(loclit_lxbol)

## Direct comparisons between original and XGA measurements

This is the the point of this notebook, making direct comparisons of like for like (or as near as we can achieve) measurements from literature to measurements made using XGA. In each case we plot simple one-to-one comparisons, with a one-to-one dashed line in red to give a reference.

### LoCuSS

We directly compare temperatures and luminosities measured by LoCuSS - specifically those measured by XMM; LoCuSS used both XMM and Chandra. We have measured more XMM $T_{\rm{X}}$ and $L_{\rm{X}}$ values for the LoCuSS sample than were originally measured, most likely due to the several years of extra data that we have access to.

In [ ]:
print('{s} out of {t} XGA Txce measurements were successful'.format(s=np.isfinite(locuss_txce[:, 0]).sum(), 
                                                                  t=len(locuss_txce)))
print('{s} out of {t} XGA Lxbolce measurements were successful'.format(s=np.isfinite(locuss_lxbol[:, 0]).sum(), 
                                                                    t=len(locuss_lxbol)))

In [ ]:
print('There were {n} LoCuSS temperatures originally from Graham'.format(n=np.isfinite(loclit_tx[:, 0]).sum()))

#### $T_{\rm{X}, 500ce}$ and $L^{\rm{bol}}_{\rm{X}, 500}$

In [ ]:
direct_comparison_plot([loclit_tx, loclit_lxbol], [locuss_txce, locuss_lxbol], 
                       [r"LoCuSS $T_{\rm{X, 500ce}}$ [keV]", r"LoCuSS $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]"], 
                       [r"XGA $T_{\rm{X, 500ce}}$ [keV]", r"XGA $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]"],
                       r"LoCuSS High-$L_{\rm{X}}$", (12, 6), ['linear', 'log'], ['linear', 'log'], 
                       savepath='../../outputs/figures/locuss_Txce_Lxbol_comp.pdf')

In [ ]:
fit_comp(loclit_tx, locuss_txce, [[0, 100], [0, 5]], r"LoCuSS $T_{\rm{X, 500ce}}$ [keV]", 
         r"XGA $T_{\rm{X, 500ce}}$ [keV]", Quantity(1, 'keV'),  num_steps=20000, view_chains=True, 
         linear=True, view_corner=True)
# fit_comp(loclit_tx, locuss_txce, [[0, 100], [0, 5]], num_steps=10000, view_chains=False)

In [ ]:
fit_comp(loclit_lxbol, locuss_lxbol, [[0, 100], [0, 5]], r"LoCuSS $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]", 
         r"XGA $L^{\rm{bol}}_{\rm{X, 500}}$ [erg s$^{-1}$]", Quantity(5e+45, 'erg/s'), num_steps=20000, 
         view_chains=True, linear=False, view_corner=True, xscale='log', yscale='log')